In [32]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
# from pywebcopy import save_webpage

In [33]:

internal_urls = set()
external_urls = set()

In [34]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [35]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [36]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_urls(url)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)


In [37]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://covid19.columbia.edu/"

In [38]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='covid19.columbia.edu', path='/', params='', query='', fragment='')
https://covid19.columbia.edu
covid19.columbia.edu/
covid19.columbia.edu


In [39]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: https://covid19.columbia.edu/
[INT] Internal link: https://covid19.columbia.edu/university-communications
[INT] Internal link: https://covid19.columbia.edu/videos
[INT] Internal link: https://covid19.columbia.edu/content/i-need-information-about
[INT] Internal link: https://covid19.columbia.edu/documents
[INT] Internal link: https://covid19.columbia.edu/content/how-you-can-help
[INT] Internal link: https://covid19.columbia.edu/content/academic-planning
[INT] Internal link: https://covid19.columbia.edu/content/academic-calendar-2020-21-0
[INT] Internal link: https://covid19.columbia.edu/content/online-teaching-toolkit
[INT] Internal link: https://covid19.columbia.edu/content/online-learning-toolkit
[INT] Internal link: https://covid19.columbia.edu/content/facilities-and-campus-life
[INT] Internal link: https://covid19.columbia.edu/content/how-campus-life-will-be-different
[INT] Internal link: https://covid19.columbia.edu/content/symptom-self-checking
[INT] Internal 

[INT] Internal link: https://covid19.columbia.edu/housing
[INT] Internal link: https://covid19.columbia.edu/online-classes
[INT] Internal link: https://covid19.columbia.edu/news/accelerated-check-out-timeline-undergraduate-residence-halls
[INT] Internal link: https://covid19.columbia.edu/news/accelerated-check-out-timeline-undergraduate-residents
[INT] Internal link: https://covid19.columbia.edu/news/all-undergraduate-students-encouraged-move-out-residence-halls
[INT] Internal link: https://covid19.columbia.edu/news/student-information-remainder-spring-semester
[INT] Internal link: https://covid19.columbia.edu/news/update-course-instruction-and-university-commencement
[INT] Internal link: https://covid19.columbia.edu/news/update-covid-19
[INT] Internal link: https://covid19.columbia.edu/commencement
[INT] Internal link: https://covid19.columbia.edu/news/medical-students-will-graduate-early
[INT] Internal link: https://covid19.columbia.edu/health-0
[INT] Internal link: https://covid19.c

In [40]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,4):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        crawl(immediate_url)
        print("[+] NEW Total External links:", len(external_urls))
        print("[+] NEW Total Internal links:", len(internal_urls))
        print("[+] NEW Total:", len(external_urls) + len(internal_urls))
        if len(internal_urls)>1000:
            break

131 0
[+] NEW Total External links: 90
[+] NEW Total Internal links: 131
[+] NEW Total: 221
[+] NEW Total External links: 90
[+] NEW Total Internal links: 131
[+] NEW Total: 221
[+] NEW Total External links: 90
[+] NEW Total Internal links: 131
[+] NEW Total: 221
[+] NEW Total External links: 94
[+] NEW Total Internal links: 131
[+] NEW Total: 225
[+] NEW Total External links: 97
[+] NEW Total Internal links: 131
[+] NEW Total: 228
[+] NEW Total External links: 97
[+] NEW Total Internal links: 131
[+] NEW Total: 228
[INT] Internal link: https://reopencu.covid19.columbia.edu/
[INT] Internal link: https://covid19.columbia.edu/content/reopencu-symptom-self-check-app
[INT] Internal link: https://covid19.columbia.edu/content/tel:2128541919
[INT] Internal link: https://covid19.columbia.edu/content/tel:2123054357
[INT] Internal link: https://reopencu.covid19.columbia.edu
[INT] Internal link: https://covid19.columbia.edu/faculty
[INT] Internal link: https://covid19.columbia.edu/staff
[INT] Int

[+] NEW Total External links: 128
[+] NEW Total Internal links: 190
[+] NEW Total: 318
[+] NEW Total External links: 128
[+] NEW Total Internal links: 190
[+] NEW Total: 318
[+] NEW Total External links: 131
[+] NEW Total Internal links: 190
[+] NEW Total: 321
[+] NEW Total External links: 131
[+] NEW Total Internal links: 190
[+] NEW Total: 321
[+] NEW Total External links: 131
[+] NEW Total Internal links: 190
[+] NEW Total: 321
[INT] Internal link: https://covid19.columbia.edu/content/ColumbiaCTL+Courseworks@columbia.edu
[+] NEW Total External links: 155
[+] NEW Total Internal links: 191
[+] NEW Total: 346
[+] NEW Total External links: 155
[+] NEW Total Internal links: 191
[+] NEW Total: 346
[+] NEW Total External links: 155
[+] NEW Total Internal links: 191
[+] NEW Total: 346
[+] NEW Total External links: 155
[+] NEW Total Internal links: 191
[+] NEW Total: 346
[+] NEW Total External links: 155
[+] NEW Total Internal links: 191
[+] NEW Total: 346
[+] NEW Total External links: 155
[

[+] NEW Total External links: 285
[+] NEW Total Internal links: 220
[+] NEW Total: 505
[INT] Internal link: https://covid19.columbia.edu/content/types-facial-personal-protective-equipment-ppe
[INT] Internal link: https://covid19.columbia.edu/content/responsibilities-returning-researchers
[INT] Internal link: https://covid19.columbia.edu/content/pilab-manager-research-laboratory-ramp-checklist
[+] NEW Total External links: 292
[+] NEW Total Internal links: 223
[+] NEW Total: 515
[+] NEW Total External links: 292
[+] NEW Total Internal links: 223
[+] NEW Total: 515
[+] NEW Total External links: 299
[+] NEW Total Internal links: 223
[+] NEW Total: 522
[+] NEW Total External links: 299
[+] NEW Total Internal links: 223
[+] NEW Total: 522
[+] NEW Total External links: 299
[+] NEW Total Internal links: 223
[+] NEW Total: 522
[+] NEW Total External links: 299
[+] NEW Total Internal links: 223
[+] NEW Total: 522
[+] NEW Total External links: 299
[+] NEW Total Internal links: 223
[+] NEW Total:

[INT] Internal link: https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/Types%20of%20Facial%20Personal%20Protective%20Equipment%2006-10-20.pdf
[INT] Internal link: https://covid19.columbia.edu/fact-sheet
[+] NEW Total External links: 329
[+] NEW Total Internal links: 281
[+] NEW Total: 610
[+] NEW Total External links: 329
[+] NEW Total Internal links: 281
[+] NEW Total: 610
[+] NEW Total External links: 329
[+] NEW Total Internal links: 281
[+] NEW Total: 610
[+] NEW Total External links: 329
[+] NEW Total Internal links: 281
[+] NEW Total: 610
[+] NEW Total External links: 329
[+] NEW Total Internal links: 281
[+] NEW Total: 610
[+] NEW Total External links: 329
[+] NEW Total Internal links: 281
[+] NEW Total: 610
[INT] Internal link: https://covid19.columbia.edu/content/getting-started
[INT] Internal link: https://covid19.columbia.edu/content/visas-and-travel
[INT] Internal link: https://covid19.columbia.edu/content/maintaining-your-status
[INT] Interna

[INT] Internal link: https://covid19.columbia.edu/sites/default/files/content/Crisis Care COVID-19_Columbia - thru June 30.pdf
[INT] Internal link: https://covid19.columbia.edu/content/bright-horizons-programs-columbia-university
[INT] Internal link: https://covid19.columbia.edu/content/parental-policies
[INT] Internal link: https://covid19.columbia.edu/content/flexible-work-arrangements
[INT] Internal link: https://covid19.columbia.edu/content/adult-and-elder-care
[+] NEW Total External links: 360
[+] NEW Total Internal links: 373
[+] NEW Total: 733
[INT] Internal link: https://covid19.columbia.edu/content/all-services
[INT] Internal link: https://covid19.columbia.edu/content/columbia-health-patient-digital-resources
[INT] Internal link: https://covid19.columbia.edu/services/individual-counseling
[INT] Internal link: https://covid19.columbia.edu/services/primary-care
[INT] Internal link: https://covid19.columbia.edu/content/support-groups
[INT] Internal link: https://covid19.columbia.

...

[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 65

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590
[+] NEW Total External links: 657
[+] NEW Total Internal links: 933
[+] NEW Total: 1590


In [41]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [42]:
semifinal_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    semifinal_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)

print(semifinal_immediate_urls)
total_count = len(semifinal_immediate_urls)
print(total_count)

{'https://covid19.columbia.edu/email-forwarding', 'https://covid19.columbia.edu/news/student-information-remainder-spring-semester', 'https://covid19.columbia.edu/content/affiliated-early-learning-centers', 'https://covid19.columbia.edu/content/types-facial-personal-protective-equipment-ppe', 'https://covid19.columbia.edu/content/body-image-concerns', 'https://covid19.columbia.edu/menulink/conflict-interest-and-research', 'https://covid19.columbia.edu/news/columbia-develops-ventilator-sharing-protocol-covid-19-patients', 'https://covid19.columbia.edu/facilities-management/covid-19/campus-housing-resources', 'https://covid19.columbia.edu/content/responsibilities-returning-researchers', 'https://covid19.columbia.edu/content/medical-services', 'https://covid19.columbia.edu/content/final-term-reminders', 'https://covid19.columbia.edu/content/back-care', 'https://covid19.columbia.edu/content/reporting-your-first-arrival-f-and-j', 'https://covid19.columbia.edu/news/student-housing-updates-de

In [43]:
final_immediate_urls = set()
for semifinal_immediate_url in semifinal_immediate_urls:
    if "tel:" not in semifinal_immediate_url:
        final_immediate_urls.add(semifinal_immediate_url)

print(final_immediate_urls)
total_count = len(final_immediate_urls)
print(total_count)       

{'https://covid19.columbia.edu/email-forwarding', 'https://covid19.columbia.edu/news/student-information-remainder-spring-semester', 'https://covid19.columbia.edu/content/affiliated-early-learning-centers', 'https://covid19.columbia.edu/content/types-facial-personal-protective-equipment-ppe', 'https://covid19.columbia.edu/content/body-image-concerns', 'https://covid19.columbia.edu/news/columbia-develops-ventilator-sharing-protocol-covid-19-patients', 'https://covid19.columbia.edu/menulink/conflict-interest-and-research', 'https://covid19.columbia.edu/facilities-management/covid-19/campus-housing-resources', 'https://covid19.columbia.edu/content/responsibilities-returning-researchers', 'https://covid19.columbia.edu/content/medical-services', 'https://covid19.columbia.edu/content/final-term-reminders', 'https://covid19.columbia.edu/content/back-care', 'https://covid19.columbia.edu/content/reporting-your-first-arrival-f-and-j', 'https://covid19.columbia.edu/news/columbia-university-press-

In [44]:

parent_folder = "Doctoral" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
    os.makedirs(f'{parent_folder}\\{name_of_folder}')

In [47]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    if ' ' in i:
        continue
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
        print(f"{code} can_save - {i}")
    except urllib.error.HTTPError as e:
        if e.code not in (200,299):
            webpage_cannot_save = webpage_cannot_save + 1
            print("discarded")
            continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        

discarded
200 can_save - https://covid19.columbia.edu/news/student-information-remainder-spring-semester
discarded
200 can_save - https://covid19.columbia.edu/content/types-facial-personal-protective-equipment-ppe
discarded
200 can_save - https://covid19.columbia.edu/news/columbia-develops-ventilator-sharing-protocol-covid-19-patients
discarded
discarded
200 can_save - https://covid19.columbia.edu/content/responsibilities-returning-researchers
discarded
discarded
200 can_save - https://covid19.columbia.edu/content/back-care
discarded
200 can_save - https://covid19.columbia.edu/news/columbia-university-press-makes-e-books-available-free
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/covid-19-health-update
discarded
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/covid-19-information-social-distancing-isolation-and-quarantine-practices
discarded
discarded
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/up

discarded
200 can_save - https://covid19.columbia.edu/events/during-after-flattening-curve-decreasing-covid-19-transmission-settings-limited-health
200 can_save - https://covid19.columbia.edu/human-resources-0
200 can_save - https://covid19.columbia.edu/news/columbia-explores-rash-coronavirus-related-bigotry
discarded
discarded
200 can_save - https://covid19.columbia.edu/events/innovation-and-resiliency-human-rights-organizations-during-pandemic
200 can_save - https://covid19.columbia.edu/graduate-students
200 can_save - https://covid19.columbia.edu/guidelines
discarded
discarded
discarded
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/columbia-college-sends-note-undergraduates
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/teachers-college-updates-plans-start-2020-21-academic-year
discarded
200 can_save - https://covid19.columbia.edu/news/acce

200 can_save - https://covid19.columbia.edu/content/about-covid-19
discarded
discarded
200 can_save - https://covid19.columbia.edu/content/discrimination-and-harassment
discarded
200 can_save - https://covid19.columbia.edu/facilities-and-campus-life
200 can_save - https://covid19.columbia.edu/news/update-coronavirus-and-travel-2120
discarded
200 can_save - https://covid19.columbia.edu/housing
discarded
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/libraries-announces-update-reopening-plans
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/mailman-school-public-heath-provides-updates-2020-21-academic-year
discarded
discarded
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/content/guidelines-planning-resumption-field-research
discarded
discarded
200 can_save - https://covid19.columbia.edu/content/er-physician-assistant-how-ppe-created-barrier-her-patients
discarded
discarded
200 can_save - https://cov

200 can_save - https://covid19.columbia.edu/fact-sheet
discarded
200 can_save - https://covid19.columbia.edu/content/finance-contingency-planning
200 can_save - https://covid19.columbia.edu/news/response-recent-immigration-restrictions
discarded
200 can_save - https://covid19.columbia.edu/content/covid-19-initial-testing-implementation-process
200 can_save - https://covid19.columbia.edu/content/facilities-and-campus-life
200 can_save - https://covid19.columbia.edu/covid-19
discarded
200 can_save - https://covid19.columbia.edu/news/dean-school-professional-studies-welcomes-new-students
200 can_save - https://covid19.columbia.edu/news/provost-announces-academic-calendar-2020-21
discarded
discarded
200 can_save - https://covid19.columbia.edu/events/frontline-nurses-leaders-pandemic-response
discarded
discarded
discarded
200 can_save - https://covid19.columbia.edu/news/how-new-york-city-phase-one-affects-university
200 can_save - https://covid19.columbia.edu/content/online-learning-toolkit

In [48]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    if webpage[-4:] == ".pdf":
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
            file_name = "Home"
        pdf_urls.add(webpage)
        print(webpage)
        r = requests.get(url, stream=True)
        with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
            for chunk in r.iter_content(2000):
                fd.write(chunk)
        pdf_count = pdf_count + 1

print(f"Pdfs saved: {pdf_count}")

https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/Health%20Screening%20Form%20for%20Service%20Personnel.pdf
https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/CU%20COVID%20Fact%20Sheet%20%234%20Face%20Coverings%20%20Masks%20April%207%202020.pdf
https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/Academic%20Calendar%202020-21.pdf
https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/CUFO_Operations_Custodial_Protocols_for_Pandemic.pdf
https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/Types%20of%20Facial%20Personal%20Protective%20Equipment%2006-10-20.pdf
https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/Interim%20Space%20Guidelines%20Assessment%20Protocol.pdf
https://covid19.columbia.edu/sites/default/files/content/Documents%20%26%20Forms/Roles%20and%20Responsibilities%20of%20Returning%20Researchers.pdf
https://

In [49]:
final_immediate_urls = final_immediate_urls - pdf_urls
webpage_save = webpage_save - pdf_urls
web_save_count = len(webpage_save)
total_count = len(final_immediate_urls)
print(total_count) 

864


In [50]:

#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

In [51]:
final_webpage_save_count = webpage_save_count - pdf_count

In [52]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    url = webpage
    file_name = url.replace(base_url,'')
    file_name = file_name.replace('/','')
    if file_name == '':
            file_name = "Home"
    urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
    webpage_save_counter_loop = webpage_save_counter_loop+1
    print(f'{webpage_save_counter_loop} of {final_webpage_save_count}  - {webpage}')

print("Saving Completed")

1 of 229  - https://covid19.columbia.edu/content/parent-resources
2 of 229  - https://covid19.columbia.edu/events/deans-seminar-series-chronic-disease-coronavirus-vaccine-prospects-and-potential-problems
3 of 229  - https://covid19.columbia.edu/
4 of 229  - https://covid19.columbia.edu/news/student-information-remainder-spring-semester
5 of 229  - https://covid19.columbia.edu/form
6 of 229  - https://covid19.columbia.edu/research
7 of 229  - https://covid19.columbia.edu/content/restroom-use-and-maintenance-bulletin
8 of 229  - https://covid19.columbia.edu/content/custodial-operations-protocol
9 of 229  - https://covid19.columbia.edu/facilities-and-campus-life-0
10 of 229  - https://covid19.columbia.edu/content/types-facial-personal-protective-equipment-ppe
11 of 229  - https://covid19.columbia.edu/news/columbia-develops-ventilator-sharing-protocol-covid-19-patients
12 of 229  - https://covid19.columbia.edu/content/are-my-symptoms-allergies-flu-or-coronavirus
13 of 229  - https://covid1

100 of 229  - https://reopencu.covid19.columbia.edu/
101 of 229  - https://covid19.columbia.edu/content/resuming-limited-site-operations-mailman-school-public-health
102 of 229  - https://covid19.columbia.edu/news/columbia-researchers-suggest-potential-late-may-covid-19-rebound
103 of 229  - https://covid19.columbia.edu/covid-19-0
104 of 229  - https://covid19.columbia.edu/content/research-ramp-framework
105 of 229  - https://covid19.columbia.edu/content/important-details-about-recovery-planning-and-ramping-site-activity
106 of 229  - https://covid19.columbia.edu/content/enhanced-university-health-and-safety-policy
107 of 229  - https://covid19.columbia.edu/documents
108 of 229  - https://covid19.columbia.edu/news/update-course-instruction-and-university-commencement
109 of 229  - https://covid19.columbia.edu/online-teaching
110 of 229  - https://covid19.columbia.edu/news/moving-forward-responsibly
111 of 229  - https://covid19.columbia.edu/news/dean-mailman-school-public-health-delive

200 of 229  - https://covid19.columbia.edu/housing-0
201 of 229  - https://covid19.columbia.edu/news/health-advisory-regarding-community-transmission-nyc
202 of 229  - https://covid19.columbia.edu/health-2
203 of 229  - https://covid19.columbia.edu/news/subway-data-reveals-communities-color-carry-burden-essential-work-and-covid-19
204 of 229  - https://covid19.columbia.edu/staff
205 of 229  - https://covid19.columbia.edu/human-resources-1
206 of 229  - https://covid19.columbia.edu/news/could-new-ultraviolet-technology-fight-spread-coronavirus
207 of 229  - https://covid19.columbia.edu/checklist
208 of 229  - https://covid19.columbia.edu/news/zuckerman-institute-finds-clever-way-keep-its-researchers-safe
209 of 229  - https://covid19.columbia.edu/health-3
210 of 229  - https://covid19.columbia.edu/news/update-columbias-response-coronavirus
211 of 229  - https://covid19.columbia.edu/news/columbia-adjunct-authors-nyu-study-ai-tool-predict-respiratory-disease-patients
212 of 229  - https:/

In [54]:
count = 0
for i in final_immediate_urls:
    url = i
    file_name = url.replace(base_url,'')
    file_name = file_name.replace('/','')
    if file_name == '':
            file_name = "Home"
    count = count + 1
    print(f"{count} of {total_count}")
    print(f'Visiting {base_url_text}/{file_name}')
    print(f'...Taking a screenshot')
    driver.get(url)
    S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
    driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
    print(f'Screenshot of {file_name} page taken! \n')
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

1 of 864
Visiting covid19.columbia.edu//email-forwarding
...Taking a screenshot
Screenshot of email-forwarding page taken! 

2 of 864
Visiting covid19.columbia.edu//newsstudent-information-remainder-spring-semester
...Taking a screenshot
Screenshot of newsstudent-information-remainder-spring-semester page taken! 

3 of 864
Visiting covid19.columbia.edu//contentaffiliated-early-learning-centers
...Taking a screenshot
Screenshot of contentaffiliated-early-learning-centers page taken! 

4 of 864
Visiting covid19.columbia.edu//contenttypes-facial-personal-protective-equipment-ppe
...Taking a screenshot
Screenshot of contenttypes-facial-personal-protective-equipment-ppe page taken! 

5 of 864
Visiting covid19.columbia.edu//contentbody-image-concerns
...Taking a screenshot
Screenshot of contentbody-image-concerns page taken! 

6 of 864
Visiting covid19.columbia.edu//newscolumbia-develops-ventilator-sharing-protocol-covid-19-patients
...Taking a screenshot
Screenshot of newscolumbia-develops-

Screenshot of COVID-19_allowability-on-sponsored-projects page taken! 

52 of 864
Visiting covid19.columbia.edu//faculty
...Taking a screenshot
Screenshot of faculty page taken! 

53 of 864
Visiting covid19.columbia.edu//science-security
...Taking a screenshot
Screenshot of science-security page taken! 

54 of 864
Visiting covid19.columbia.edu//contenttransferring-your-j-1-sevis-record-another-school
...Taking a screenshot
Screenshot of contenttransferring-your-j-1-sevis-record-another-school page taken! 

55 of 864
Visiting covid19.columbia.edu//contentmeet-scholar-team
...Taking a screenshot
Screenshot of contentmeet-scholar-team page taken! 

56 of 864
Visiting covid19.columbia.edu//contenttell-us-what-you-think
...Taking a screenshot
Screenshot of contenttell-us-what-you-think page taken! 

57 of 864
Visiting covid19.columbia.edu//menulinknew-york-state-agencies
...Taking a screenshot
Screenshot of menulinknew-york-state-agencies page taken! 

58 of 864
Visiting covid19.columbia.ed

Screenshot of contentconsulting-services page taken! 

104 of 864
Visiting covid19.columbia.edu//newsbarnard-announces-academic-calendar-2020-21
...Taking a screenshot
Screenshot of newsbarnard-announces-academic-calendar-2020-21 page taken! 

105 of 864
Visiting covid19.columbia.edu//office-postdoctoral-affairs
...Taking a screenshot
Screenshot of office-postdoctoral-affairs page taken! 

106 of 864
Visiting covid19.columbia.edu//contenttaxes
...Taking a screenshot
Screenshot of contenttaxes page taken! 

107 of 864
Visiting covid19.columbia.edu//contentuse-and-management-controlled-substances
...Taking a screenshot
Screenshot of contentuse-and-management-controlled-substances page taken! 

108 of 864
Visiting covid19.columbia.edu//eventsuniversity-life-forum-your-questions-answered-covid-19
...Taking a screenshot
Screenshot of eventsuniversity-life-forum-your-questions-answered-covid-19 page taken! 

109 of 864
Visiting covid19.columbia.edu//COVID-19_Facilities-Support-of-Research
..

Screenshot of contentB-1_B-2_ESTA page taken! 

156 of 864
Visiting covid19.columbia.edu//ocha
...Taking a screenshot
Screenshot of ocha page taken! 

157 of 864
Visiting covid19.columbia.edu//contentacademic-calendar-2020-21-0
...Taking a screenshot
Screenshot of contentacademic-calendar-2020-21-0 page taken! 

158 of 864
Visiting covid19.columbia.edu//contentadministrative-hiring-freeze-exception-request-process
...Taking a screenshot
Screenshot of contentadministrative-hiring-freeze-exception-request-process page taken! 

159 of 864
Visiting covid19.columbia.edu//contentemergency-contacts
...Taking a screenshot
Screenshot of contentemergency-contacts page taken! 

160 of 864
Visiting covid19.columbia.edu//contentsponsoring-or-extending-h-1b
...Taking a screenshot
Screenshot of contentsponsoring-or-extending-h-1b page taken! 

161 of 864
Visiting covid19.columbia.edu//contentapplying-your-i-20ds-2019
...Taking a screenshot
Screenshot of contentapplying-your-i-20ds-2019 page taken! 



Screenshot of newsfrequently-asked-questions-faq-how-administrative-mail-operating-during-covid-19 page taken! 

211 of 864
Visiting covid19.columbia.edu//menulinkoffice-executive-vice-president-research
...Taking a screenshot
Screenshot of menulinkoffice-executive-vice-president-research page taken! 

212 of 864
Visiting covid19.columbia.edu//contentonline-learning
...Taking a screenshot
Screenshot of contentonline-learning page taken! 

213 of 864
Visiting covid19.columbia.edu//app
...Taking a screenshot
Screenshot of app page taken! 

214 of 864
Visiting covid19.columbia.edu//contentadmissions-diagnostic-testing
...Taking a screenshot
Screenshot of contentadmissions-diagnostic-testing page taken! 

215 of 864
Visiting covid19.columbia.edu//eventsdeans-seminar-series-chronic-disease-coronavirus-vaccine-prospects-and-potential-problems
...Taking a screenshot
Screenshot of eventsdeans-seminar-series-chronic-disease-coronavirus-vaccine-prospects-and-potential-problems page taken! 

216 

...

Screenshot of librariesmusic.html page taken! 

631 of 864
Visiting covid19.columbia.edu//coronavirus
...Taking a screenshot
Screenshot of coronavirus page taken! 

632 of 864
Visiting covid19.columbia.edu//announcement-0
...Taking a screenshot
Screenshot of announcement-0 page taken! 

633 of 864
Visiting covid19.columbia.edu//finance
...Taking a screenshot
Screenshot of finance page taken! 

634 of 864
Visiting covid19.columbia.edu//internal-controls
...Taking a screenshot
Screenshot of internal-controls page taken! 

635 of 864
Visiting covid19.columbia.edu//contentopt-reporting
...Taking a screenshot
Screenshot of contentopt-reporting page taken! 

636 of 864
Visiting covid19.columbia.edu//contentabout-human-resources
...Taking a screenshot
Screenshot of contentabout-human-resources page taken! 

637 of 864
Visiting covid19.columbia.edu//contentenforcement-plan-enhanced-health-and-safety-policy
...Taking a screenshot
Screenshot of contentenforcement-plan-enhanced-health-and-safety-

Screenshot of contentinterim-space-use-and-modification-guidelines page taken! 

686 of 864
Visiting covid19.columbia.edu//contentanxiety
...Taking a screenshot
Screenshot of contentanxiety page taken! 

687 of 864
Visiting covid19.columbia.edu//newstransaction-processing-updates-and-reminders
...Taking a screenshot
Screenshot of newstransaction-processing-updates-and-reminders page taken! 

688 of 864
Visiting covid19.columbia.edu//newsfall-classes-institute-human-nutrition-will-be-online
...Taking a screenshot
Screenshot of newsfall-classes-institute-human-nutrition-will-be-online page taken! 

689 of 864
Visiting covid19.columbia.edu//newspac-transaction-process-reminders
...Taking a screenshot
Screenshot of newspac-transaction-process-reminders page taken! 

690 of 864
Visiting covid19.columbia.edu//contentfacts-about-ebola
...Taking a screenshot
Screenshot of contentfacts-about-ebola page taken! 

691 of 864
Visiting covid19.columbia.edu//contentmanage-salary-and-pay
...Taking a s

Screenshot of newsramping-laboratory-and-clinical-research page taken! 

739 of 864
Visiting covid19.columbia.edu//exploring-1619
...Taking a screenshot
Screenshot of exploring-1619 page taken! 

740 of 864
Visiting covid19.columbia.edu//researchers-resources
...Taking a screenshot
Screenshot of researchers-resources page taken! 

741 of 864
Visiting covid19.columbia.edu//menulinknih-loan-repayment-program-lrp
...Taking a screenshot
Screenshot of menulinknih-loan-repayment-program-lrp page taken! 

742 of 864
Visiting covid19.columbia.edu//academics
...Taking a screenshot
Screenshot of academics page taken! 

743 of 864
Visiting covid19.columbia.edu//formsccss-consultation-request-form
...Taking a screenshot
Screenshot of formsccss-consultation-request-form page taken! 

744 of 864
Visiting covid19.columbia.edu//contentroommates
...Taking a screenshot
Screenshot of contentroommates page taken! 

745 of 864
Visiting covid19.columbia.edu//newscolumbia-opens-two-additional-residence-halls

Screenshot of menulinkfederal-agencies page taken! 

790 of 864
Visiting covid19.columbia.edu//contentlibraryweb.html
...Taking a screenshot
Screenshot of contentlibraryweb.html page taken! 

791 of 864
Visiting covid19.columbia.edu//contentdepartment-responsibilities
...Taking a screenshot
Screenshot of contentdepartment-responsibilities page taken! 

792 of 864
Visiting covid19.columbia.edu//faqs
...Taking a screenshot
Screenshot of faqs page taken! 

793 of 864
Visiting covid19.columbia.edu//contentdining
...Taking a screenshot
Screenshot of contentdining page taken! 

794 of 864
Visiting covid19.columbia.edu//contentisso-covid-19-student-faqs
...Taking a screenshot
Screenshot of contentisso-covid-19-student-faqs page taken! 

795 of 864
Visiting covid19.columbia.edu//academic-planning-1
...Taking a screenshot
Screenshot of academic-planning-1 page taken! 

796 of 864
Visiting covid19.columbia.edu//contentmoving-guide
...Taking a screenshot
Screenshot of contentmoving-guide page tak

Screenshot of menulinkdirectory page taken! 

841 of 864
Visiting covid19.columbia.edu//student-lifeDACA-admissions-information-prospective-students
...Taking a screenshot
Screenshot of student-lifeDACA-admissions-information-prospective-students page taken! 

842 of 864
Visiting covid19.columbia.edu//newsfree-meals-new-yorkers
...Taking a screenshot
Screenshot of newsfree-meals-new-yorkers page taken! 

843 of 864
Visiting covid19.columbia.edu//daca-undocumented-students
...Taking a screenshot
Screenshot of daca-undocumented-students page taken! 

844 of 864
Visiting covid19.columbia.edu//contentmaintaining-your-status
...Taking a screenshot
Screenshot of contentmaintaining-your-status page taken! 

845 of 864
Visiting covid19.columbia.edu//aboutawardsbancroft.html
...Taking a screenshot
Screenshot of aboutawardsbancroft.html page taken! 

846 of 864
Visiting covid19.columbia.edu//well-being
...Taking a screenshot
Screenshot of well-being page taken! 

847 of 864
Visiting covid19.colu